In [1]:
# Install required packages
%pip install sympy langchain panel param python-dotenv wikipedia-api openai requests


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 45.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 76.9 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=1f171ce43292c9aebd58c83a481ce839799796fa3125bf6510aff99c62bd72e8
  Stored in directory: /home/jovyan/.cache/pip/wheels/3c/27/86/6352987e263844a6793aef9a0485e9ebbf8b93cd1f9b020563
Successfully built wikipedia-api


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import openai
import panel as pn
import param
import sympy as sp
import json
import requests
from dotenv import load_dotenv, find_dotenv

from langchain.tools import tool
from langchain.tools.render import format_tool_to_openai_function  # Corrected import
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

pn.extension()

# Load OpenAI API key from environment
_ = load_dotenv(find_dotenv())  # Read local .env file
openai.api_key = os.environ.get('OPENAI_API_KEY')


In [17]:
import json
from pydantic import BaseModel, Field, root_validator
from typing import List, Dict, Union

# Define the path to your JSON file
json_file_path = 'student-data-Final.json'

# Load the student data from the JSON file
with open(json_file_path, 'r') as file:
    student_data = json.load(file)

# Define Models for Validation
class Attendance(BaseModel):
    total_classes: int
    classes_attended: int
    attendance_percentage: str

class SubjectScores(BaseModel):
    score: str
    total_questions: int
    correct_answers: int
    incorrect_answers: int
    unattempted_questions: int
    topics_struggled: List[str]
    topics_mastered: List[str]
    time_spent_minutes: int
    attendance: Attendance

class StudentPersona(BaseModel):
    student_id: str
    name: str
    class_: str = Field(..., alias="class")  # Use alias for reserved keyword
    exam_type: str
    overall_percentage: int

class TestScores(BaseModel):
    overall_score: str
    test_percentage: str
    total_questions: int
    correct_answers: int
    incorrect_answers: int
    unattempted_questions: int

class StudentDataInput(BaseModel):
    student_persona: StudentPersona
    test_scores: TestScores
    subject_scores_physics: SubjectScores
    subject_scores_chemistry: SubjectScores
    subject_scores_biology: SubjectScores
    overall_attendance: Attendance

# Tool to analyze student data
@tool(args_schema=StudentDataInput)
def analyze_student_data(student_data: StudentDataInput) -> str:
    """Analyze the student data, find weaker subjects, mastered topics, and attendance"""
    try:
        # Accessing data from the StudentDataInput model
        name = student_data.student_persona.name
        
        # Collect weaker subjects
        weaker_subjects = []
        if int(student_data.subject_scores_physics.score.split()[0]) < 40:
            weaker_subjects.append("Physics")
        if int(student_data.subject_scores_chemistry.score.split()[0]) < 40:
            weaker_subjects.append("Chemistry")
        if int(student_data.subject_scores_biology.score.split()[0]) < 40:
            weaker_subjects.append("Biology")

        # Collect mastered topics
        mastered_topics = {
            "Physics": student_data.subject_scores_physics.topics_mastered,
            "Chemistry": student_data.subject_scores_chemistry.topics_mastered,
            "Biology": student_data.subject_scores_biology.topics_mastered
        }

        # Check overall attendance
        overall_attendance = student_data.overall_attendance.attendance_percentage

        # Generate advice based on the analysis
        response = f"Hello {name}, here is your performance analysis:\n\n"

        if weaker_subjects:
            response += f"You're struggling with the following subjects: {', '.join(weaker_subjects)}.\n"
        else:
            response += "You're performing well in all subjects.\n"
        
        response += "Mastered topics:\n"
        for subject, topics in mastered_topics.items():
            response += f"- {subject}: {', '.join(topics)}\n"

        response += f"Your overall attendance is {overall_attendance}. "
        if float(overall_attendance.strip('%')) < 50:
            response += "You should improve your attendance to boost your performance."

        return response
    except Exception as e:
        return f"An error occurred while analyzing the student data: {str(e)}"

# Convert raw data to Pydantic model
student_data_model = StudentDataInput(**student_data)

# Test the tool
response = analyze_student_data(student_data_model)
print(response)


ValidationError: 3 validation errors for StudentDataInput
subject_scores_physics -> attendance
  field required (type=value_error.missing)
subject_scores_chemistry -> attendance
  field required (type=value_error.missing)
subject_scores_biology -> attendance
  field required (type=value_error.missing)

In [3]:
class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_tool_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

    def clr_history(self, count=0):
        self.chat_history = []
        return 


In [4]:
# Initialize the chatbot
cb = cbfs(tools)

# Set up input widget for the conversation
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Bind conversation function
conversation = pn.bind(cb.convchain, inp) 

# Create the dashboard
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Mathematical_QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)

# Display the dashboard
dashboard.servable()


Traceback (most recent call last):
 File "/usr/local/lib/python3.9/site-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.9/site-packages/panel/viewable.py", line 471, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.9/site-packages/bokeh/document/document.py", line 758, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.9/site-packages/panel/reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.9/site-packages/panel/io/state.py", line 432, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.9/site-packages/panel/reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.9/site-packages/panel/reactive.py", line 449, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.9/site-packages/panel/reactive.py", line 445, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.9/site-packages/panel/reactive.py", line 384, in _process_events
 self.param.update(**self_events)
 File "/usr/local/lib/python3.9/site-packages/param/parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.9/site-packages/param/parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.9/site-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.9/site-packages/panel/param.py", line 840, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.9/site-packages/panel/param.py", line 806, in eval
 return eval_function(function)
 File "/usr/local/lib/python3.9/site-packages/panel/util/__init__.py", line 325, in eval_function
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.9/site-packages/param/parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "/usr/local/lib/python3.9/site-packages/panel/depends.py", line 249, in wrapped
 return eval_fn()(*combined_args, **combined_kwargs)
 File "/tmp/ipykernel_111/3207255696.py", line 24, in convchain
 result = self.qa.invoke({"input": query})
 File "/usr/local/lib/python3.9/site-packages/langchain/chains/base.py", line 84, in invoke
 return self(
 File "/usr/local/lib/python3.9/site-packages/langchain/chains/base.py", line 306, in __call__
 raise e
 File "/usr/local/lib/python3.9/site-packages/langchain/chains/base.py", line 300, in __call__
 self._call(inputs, run_manager=run_manager)
 File "/usr/local/lib/python3.9/site-packages/langchain/agents/agent.py", line 1141, in _call
 next_step_output = self._take_next_step(
 File "/usr/local/lib/python3.9/site-packages/langchain/agents/agent.py", line 928, in _take_next_step
 output = self.agent.plan(
 File "/usr/local/lib/python3.9/site-packages/langchain/agents/agent.py", line 370, in plan
 output = self.runnable.invoke(inputs, config={"callbacks": callbacks})
 File "/usr/local/lib/python3.9/site-packages/langchain/schema/runnable/base.py", line 1113, in invoke
 input = step.invoke(
 File "/usr/local/lib/python3.9/site-packages/langchain/schema/runnable/passthrough.py", line 233, in invoke
 **self.mapper.invoke(input, config, **kwargs),
 File "/usr/local/lib/python3.9/site-packages/langchain/schema/runnable/base.py", line 1623, in invoke
 output = {key: future.result() for key, future in zip(steps, fut

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, height=400, loading_indicator=True)
            [3] Divider()

In [5]:
def format_tool_to_openai_functions(intermediate_steps):
    # Assuming intermediate_steps is a list of actions
    # Here we simply format them for display/logging purposes
    formatted_output = []
    for step in intermediate_steps:
        formatted_output.append({
            "tool": step[0],
            "input": step[1],
        })
    return formatted_output


In [6]:
###  pip install --upgrade langchain


In [14]:
import json

# Define the path to your JSON file
json_file_path = 'student-data-Final.json'

# Load the student data from the JSON file
with open(json_file_path, 'r') as file:
    student_data = json.load(file)

# Define the score threshold
threshold = 40

# Function to check scores and trigger chatbot
def check_scores_and_trigger_chatbot():
    name = student_data['student_persona']['name']
    
    # Extract scores for each subject
    subject_scores = {
        "Physics": int(student_data['subject_scores_physics']['score'].split()[0]),
        "Chemistry": int(student_data['subject_scores_chemistry']['score'].split()[0]),
        "Biology": int(student_data['subject_scores_biology']['score'].split()[0])
    }

    # Check scores for each subject and trigger chatbot
    for subject, score in subject_scores.items():
        if score < threshold:
            print(f"Triggering chatbot for {name} in {subject} (Score: {score})")
            # Simulate chatbot conversation
            print(f"Hi {name}, I noticed your score in {subject} is {score}. How can I help you improve?")
            # For actual chatbot functionality, replace the above line with:
            # cb.convchain(f"Hi {name}, I noticed your score in {subject} is {score}. How can I help you improve?")

# Run the check and trigger
check_scores_and_trigger_chatbot()
dashboard

Triggering chatbot for Abhinay in Biology (Score: 35)
Hi Abhinay, I noticed your score in Biology is 35. How can I help you improve?


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…', value_input='how can i improve m...)
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, height=400, loading_indicator=True)
            [3] Divider()

In [13]:
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…', value_input='how can i improve m...)
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, height=400, loading_indicator=True)
            [3] Divider()